In [16]:
import os
import torch
import torch.nn.functional as F
from transformers import GPT2Tokenizer, GPT2LMHeadModel, GPT2Config,AutoModelForCausalLM
import numpy as np
path = '../HULK/Saved_models/'

In [ ]:
conditioned_tokens = []
generated_tokens = []
past = None
fst = True
DELIMITER = "/n"

def reinput(user_msg):
    global conditioned_tokens; global fst
    os.system('cls' if os.name == 'nt' else 'clear')

    conditioned_tokens = tokenizer.decode(conditioned_tokens)

    print("Me: " + user_msg + "\n" + "Bot: ",end='')

    if fst:
      fst = False
      user_msg = "<|endoftext|>" + user_msg

    user_msg = "" + user_msg
    conditioned_tokens += user_msg 
    conditioned_tokens = tokenizer.encode(conditioned_tokens) 
    conditioned_tokens += [50256] # Append operator to prepend conversation history



def top_p_filtering(logits, top_p=0.9, filter_value=-float('Inf')):
  """
  Credit: https://gist.github.com/thomwolf/1a5a29f6962089e871b94cbd09daf317
  """
  assert logits.dim() == 1  # batch size 1 for single word generation
  sorted_logits, sorted_indices = torch.sort(logits, descending=True)
  cumulative_probs = torch.cumsum(F.softmax(sorted_logits, dim=-1), dim=-1)
  # remove tokens with cumulative probability above the threshold
  sorted_indices_to_remove = cumulative_probs > top_p
  # shift the indices to the right to keep also the first token above the threshold
  sorted_indices_to_remove[..., 1:] = sorted_indices_to_remove[..., :-1].clone()
  sorted_indices_to_remove[..., 0] = 0
  indices_to_remove = sorted_indices[sorted_indices_to_remove]
  logits[indices_to_remove] = filter_value
  return logits


def recalc(p=None):
    global conditioned_tokens
    global generated_tokens
    global past

    # for segment display purpose, keep 2 sets of tokens
    indexed_tokens = conditioned_tokens + generated_tokens

    tokens_tensor = torch.tensor([indexed_tokens])

    #tokens_tensor = tokens_tensor
    with torch.no_grad():
        outputs = model(tokens_tensor)
        predictions = outputs[0]
    logits = predictions[0, -1, :]
    filtered_logits = top_p_filtering(logits)
    probabilities = F.softmax(filtered_logits, dim=-1)
    next_token = torch.multinomial(probabilities, 1)
    generated_tokens.append(next_token.item())
    return next_token.item()

def generate():
    global conditioned_tokens
    global generated_tokens
    global past

    while True:
        if len(tokenizer.decode(conditioned_tokens)) > 320:
          dc = tokenizer.decode(conditioned_tokens)
          dc = dc[len(dc)-320:]
          idx = dc.find(DELIMITER)
          if idx != -1:
            dc = dc[idx+len(DELIMITER):]
            conditioned_tokens = tokenizer.encode(dc)

        result = recalc()

        if result == 50256:

      # end-of-text : 50256
      # use this special token to split segments

            decoded_reply = tokenizer.decode(generated_tokens)

            to_print = decoded_reply
            if to_print.endswith("<|endoftext|>"):
              to_print = to_print[:-len("<|endoftext|>")]
              decoded_reply = to_print
            print(to_print)

            decoded_reply = "" + decoded_reply
            decoded_reply = decoded_reply+DELIMITER
            conditioned_tokens += (tokenizer.encode(decoded_reply))

            # Remove end of text tokens from feeding back into model
      # -- see here for reason https://github.com/huggingface/transformers/issues/429#issuecomment-479380117
            cond_str = tokenizer.decode(conditioned_tokens)
            cond_str = cond_str.replace("<|endoftext|>",DELIMITER) # Since "<end-of-text> (50256)" is mapped to the newline character (\n) by default when calling decode():
            conditioned_tokens = tokenizer.encode(cond_str)

    # Uncomment to debug (print) conversation history that is re-fed to the model
    # cond_str = tokenizer.decode(conditioned_tokens)
    # cond_str = cond_str.replace("<|endoftext_|>","\n").replace(DELIMITER,"\n") # Since "<end-of-text> (50256)" is mapped to the newline character (\n) by default when calling decode():
    # print("(condstr_dbg) " + cond_str)
    # print(r"(/constr_dbg)")

            generated_tokens = []
            break

In [ ]:
from IPython.display import clear_output
import logging

from IPython.display import clear_output
import logging

def new_dialogue():

  global conditioned_tokens
  global generated_tokens
  global past; global fst
  
  conditioned_tokens = []; generated_tokens = []; past = None; fst = True
  reinput("What is the meaning of life?")
  generate()

  while True:
    cmd = input()

    if cmd == "reset":
      clear_output()
      new_dialogue()
      break

    if cmd != "":
      reinput(cmd)
    generate()
    

In [61]:
from transformers import AutoModelWithLMHead, AutoTokenizer,AutoModelForCausalLM
import torch

tokenizer = AutoTokenizer.from_pretrained("../HULK/Counterspeech/Saved_Models/Generator/CONAN_DialoGPT-medium/",cache_dir=path)
model = AutoModelForCausalLM.from_pretrained("../HULK/Counterspeech/Saved_Models/Generator/CONAN_DialoGPT-medium/",cache_dir=path)

In [53]:
# tokenizer = AutoTokenizer.from_pretrained("gpt2-medium",cache_dir=path)
# model = AutoModelForCausalLM.from_pretrained("gpt2-medium",cache_dir=path)

In [64]:
sentence = "Lets deport the blacks to their country <|endoftext|>"
input_ids = tokenizer.encode(sentence,truncation=True,max_length=256,return_tensors='pt') 

####### greedy_Decoding ######
beam_outputs = model.generate(

    input_ids, 
    pad_token_id         = tokenizer.eos_token_id,
    max_length           = 100,
    min_length           = 50,
    top_k                = 1000,
    top_p                = 0.92,
    repetition_penalty   = 3.5,
    temperature          = 2.0,
#             num_beams            = args['num_beans'], 
     no_repeat_ngram_size =3,  
#             early_stopping       = args['early_stopping']
)
reply = tokenizer.decode(beam_outputs[0])
print(reply)

Lets deport the blacks to their country <|endoftext|>i think you mean muslims. they are not a race. but i do agree that this is an awful idea. we should stop accepting people from all walks of life, including those that are different from us.<|endoftext|>


In [13]:
beam_outputs

tensor([[ 5492,   220,  1312,   262,   479,   288,   366,   257,   275,   517,
           649,  3381,  1271,   717,  1468,   532,   530,  1353,   355,  1267,
         50256]])

In [22]:
input_ids

tensor([[5492]])

In [33]:
inputs=[425, 925, 428, 6576, 3912, 780, 545, 262, 1611, 286, 34249, 396, 326, 722,
8, 257, 3912, 673, 7832, 290, 1838, 340, 625, 290, 625, 290, 1312, 892, 26,
2, 5103, 319, 428, 6576, 318, 617, 286, 262, 1266, 220, 425, 1683, 1760, 5,
23, 4203, 2495, 10607, 351, 3589, 546, 326]

inputs = [72, 716, 1682, 4203, 2495, 9480, 3608, 290, 7723, 2845, 329, 262, 7009, 2,300, 286, 19553, 508, 1394, 4917, 511, 835, 656, 616, 2156]    

In [34]:
tokenizer.convert_ids_to_tokens(inputs)

['i',
 'Ġam',
 'Ġactually',
 'Ġfeeling',
 'Ġpretty',
 'Ġcalm',
 'Ġcool',
 'Ġand',
 'Ġcollected',
 'Ġexcept',
 'Ġfor',
 'Ġthe',
 'Ġtiny',
 '#',
 'Ġl',
 'Ġof',
 'Ġbats',
 'Ġwho',
 'Ġkeep',
 'Ġfinding',
 'Ġtheir',
 'Ġway',
 'Ġinto',
 'Ġmy',
 'Ġhouse']